# Demo: Evaluation Metrics cho Vietnamese Cultural VQA



Notebook này demo cách sử dụng 2 metrics đánh giá:
1. **BERTScore** - Semantic similarity
2. **LLM-as-a-Judge** - Expert scoring với Gemini API

## 1. Setup

In [ ]:
# Install dependencies (uncomment nếu chưa cài)
# !pip install bert-score google-generativeai numpy tqdm

In [ ]:
import os
import json
from evaluation_metrics import (
    BERTScoreEvaluator,
    LLMJudgeEvaluator,
    VQAEvaluator
)

# Set API key (QUAN TRỌNG!)
# Lấy API key miễn phí tại: https://makersuite.google.com/app/apikey
os.environ["GEMINI_API_KEY"] = "YOUR_API_KEY_HERE"  

## 2. Demo BERTScore

In [ ]:
# Initialize BERTScore evaluator
bert_evaluator = BERTScoreEvaluator(
    model_name="bert-base-multilingual-cased",
    device="cuda"  # Đổi sang "cpu" nếu không có GPU
)

In [ ]:
# Test case 1: Câu trả lời gần giống
prediction1 = "Chùa Một Cột được xây dựng vào năm 1049 dưới triều Lý."
ground_truth1 = "Chùa Một Cột, triều Lý, năm 1049."

score1 = bert_evaluator.evaluate_single(prediction1, ground_truth1)
print(f"Test 1 - Câu trả lời tốt:")
print(f"  Prediction: {prediction1}")
print(f"  Ground Truth: {ground_truth1}")
print(f"  BERTScore: {score1:.4f}\n")

In [ ]:
# Test case 2: Câu trả lời sai
prediction2 = "Đây là Văn Miếu, được xây dựng năm 1070."
ground_truth2 = "Chùa Một Cột, triều Lý, năm 1049."

score2 = bert_evaluator.evaluate_single(prediction2, ground_truth2)
print(f"Test 2 - Câu trả lời sai:")
print(f"  Prediction: {prediction2}")
print(f"  Ground Truth: {ground_truth2}")
print(f"  BERTScore: {score2:.4f}\n")

In [ ]:
# Test batch evaluation
predictions = [
    "Vịnh Hạ Long là di sản thiên nhiên thế giới",
    "Phố cổ Hội An được UNESCO công nhận năm 1999",
    "Văn Miếu là trường đại học đầu tiên của Việt Nam"
]

references = [
    "Vịnh Hạ Long được UNESCO công nhận là di sản thế giới năm 1994",
    "Hội An là di sản văn hóa thế giới từ năm 1999",
    "Văn Miếu - Quốc Tử Giám, trường đại học đầu tiên VN, thành lập 1076"
]

scores = bert_evaluator.evaluate_batch(predictions, references)
stats = bert_evaluator.get_statistics(scores)

print("Batch Evaluation Results:")
for i, (pred, ref, score) in enumerate(zip(predictions, references, scores), 1):
    print(f"\n{i}. Score: {score:.4f}")
    print(f"   Pred: {pred}")
    print(f"   Ref:  {ref}")

print(f"\nStatistics:")
print(f"  Mean: {stats['mean']:.4f}")
print(f"  Std:  {stats['std']:.4f}")

## 3. Demo LLM-as-a-Judge

In [ ]:
# Initialize LLM Judge
llm_evaluator = LLMJudgeEvaluator(
    api_key=None,  # Will use GEMINI_API_KEY from environment
    model_name="gemini-1.5-flash"  # Free tier model
)

In [ ]:
# Test case 1: Câu trả lời xuất sắc
question1 = "Địa điểm trong ảnh có phải di sản thiên nhiên thế giới không?"
prediction1 = "Có, đây là Vịnh Hạ Long, một di sản thiên nhiên thế giới được UNESCO công nhận năm 1994."
ground_truth1 = "Có, Vịnh Hạ Long là di sản thiên nhiên thế giới, được UNESCO công nhận năm 1994."

score1, reasoning1 = llm_evaluator.evaluate_single(question1, prediction1, ground_truth1)

print("=" * 80)
print("Test 1: Câu trả lời xuất sắc")
print("=" * 80)
print(f"Câu hỏi: {question1}")
print(f"\nDự đoán: {prediction1}")
print(f"\nGround Truth: {ground_truth1}")
print(f"\n📊 Điểm: {score1}/5")
print(f"💭 Lý do: {reasoning1}")
print()

In [ ]:
# Test case 2: Câu trả lời có hallucination
question2 = "Chùa Một Cột được xây dựng vào thời nào?"
prediction2 = "Chùa Một Cột được xây dựng vào năm 1076 dưới thời vua Lý Nhân Tông."
ground_truth2 = "Chùa Một Cột được xây dựng năm 1049 dưới thời vua Lý Thái Tông."

score2, reasoning2 = llm_evaluator.evaluate_single(question2, prediction2, ground_truth2)

print("=" * 80)
print("Test 2: Câu trả lời có sai sót (hallucination)")
print("=" * 80)
print(f"Câu hỏi: {question2}")
print(f"\nDự đoán: {prediction2}")
print(f"\nGround Truth: {ground_truth2}")
print(f"\n📊 Điểm: {score2}/5")
print(f"💭 Lý do: {reasoning2}")
print()

## 4. Demo Combined Evaluator

In [ ]:
# Initialize combined evaluator
evaluator = VQAEvaluator(
    use_bert_score=True,
    use_llm_judge=True,
    gemini_api_key=None,  # Use env var
    bert_model="bert-base-multilingual-cased",
    device="cuda"
)

In [ ]:
# Evaluate single sample
result = evaluator.evaluate_single(
    image_id="001234",
    question="Đây là di tích gì?",
    prediction="Đây là Văn Miếu - Quốc Tử Giám, được xây dựng năm 1070.",
    ground_truth="Văn Miếu - Quốc Tử Giám, thành lập năm 1070 dưới triều Lý."
)

print("=" * 80)
print("EVALUATION RESULT")
print("=" * 80)
print(f"Image ID: {result.image_id}")
print(f"Question: {result.question}")
print(f"\nPredicted: {result.predicted_answer}")
print(f"\nGround Truth: {result.ground_truth}")
print(f"\n📊 Metrics:")
print(f"  - BERTScore: {result.bert_score:.4f}")
print(f"  - LLM Judge: {result.llm_judge_score}/5")
print(f"\n💭 LLM Reasoning: {result.llm_judge_reasoning}")

## 5. Evaluate Full Dataset

Để chạy trên toàn bộ dataset, cần chuẩn bị file JSON với format:
```json
[
  {
    "image_id": "001234",
    "question": "...",
    "prediction": "...",
    "ground_truth": "..."
  }
]
```

In [ ]:
# Create sample predictions file for demo
sample_data = [
    {
        "image_id": "001234",
        "question": "Đây là di tích gì?",
        "prediction": "Chùa Một Cột được xây dựng năm 1049 dưới triều Lý.",
        "ground_truth": "Chùa Một Cột, triều Lý, năm 1049."
    },
    {
        "image_id": "001235",
        "question": "Địa điểm này có phải di sản thế giới không?",
        "prediction": "Có, Vịnh Hạ Long là di sản thiên nhiên thế giới.",
        "ground_truth": "Có, được UNESCO công nhận năm 1994."
    },
    {
        "image_id": "001236",
        "question": "Văn Miếu được xây dựng khi nào?",
        "prediction": "Văn Miếu được thành lập năm 1070.",
        "ground_truth": "Văn Miếu - Quốc Tử Giám thành lập năm 1070 dưới triều Lý."
    }
]

# Save to file
with open("sample_predictions.json", "w", encoding="utf-8") as f:
    json.dump(sample_data, f, ensure_ascii=False, indent=2)

print("✓ Created sample_predictions.json")

In [ ]:
# Evaluate entire dataset
stats = evaluator.evaluate_dataset(
    predictions_file="sample_predictions.json",
    output_file="evaluation_results.json",
    llm_judge_delay=2.0  # 2 seconds delay to avoid rate limit
)

print("\n" + "="*80)
print("FINAL STATISTICS")
print("="*80)
print(json.dumps(stats, indent=2, ensure_ascii=False))

## 6. Visualize Results

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Load results
with open("evaluation_results.json", "r", encoding="utf-8") as f:
    results_data = json.load(f)

# Extract scores
bert_scores = [r["bert_score"] for r in results_data["results"] if r["bert_score"] is not None]
llm_scores = [r["llm_judge_score"] for r in results_data["results"] if r["llm_judge_score"] is not None]

# Create plots
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# BERTScore distribution
axes[0].hist(bert_scores, bins=20, color='skyblue', edgecolor='black')
axes[0].axvline(results_data["statistics"]["bert_score"]["mean"], 
                color='red', linestyle='--', label=f'Mean: {results_data["statistics"]["bert_score"]["mean"]:.4f}')
axes[0].set_xlabel('BERTScore')
axes[0].set_ylabel('Frequency')
axes[0].set_title('BERTScore Distribution')
axes[0].legend()
axes[0].grid(alpha=0.3)

# LLM Judge distribution
axes[1].hist(llm_scores, bins=5, color='lightgreen', edgecolor='black', range=(1, 5))
axes[1].axvline(results_data["statistics"]["llm_judge"]["mean"], 
                color='red', linestyle='--', label=f'Mean: {results_data["statistics"]["llm_judge"]["mean"]:.2f}')
axes[1].set_xlabel('LLM Judge Score')
axes[1].set_ylabel('Frequency')
axes[1].set_title('LLM Judge Score Distribution')
axes[1].set_xticks([1, 2, 3, 4, 5])
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('evaluation_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved visualization to evaluation_distributions.png")

In [ ]:
# Correlation between BERTScore and LLM Judge
plt.figure(figsize=(8, 6))
plt.scatter(bert_scores, llm_scores, alpha=0.6)
plt.xlabel('BERTScore')
plt.ylabel('LLM Judge Score')
plt.title('Correlation: BERTScore vs LLM Judge')
plt.grid(alpha=0.3)

# Add correlation coefficient
correlation = np.corrcoef(bert_scores, llm_scores)[0, 1]
plt.text(0.05, 4.8, f'Correlation: {correlation:.3f}', 
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.savefig('score_correlation.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved correlation plot to score_correlation.png")